In [ ]:
import csv
import torch
import numpy as np
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

if torch.cuda.is_available():
    print(f"Usando la GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA no está disponible. El modelo se ejecutará en la CPU.")


ruta_entrenamiento = '/content/drive/MyDrive/PLN/PoliticES_train_kaggle.csv'
ruta_prueba = '/content/drive/MyDrive/PLN/PoliticES_test_kaggle.csv'
archivo_predicciones = '/content/drive/MyDrive/PLN/predicciones.csv'


model_name = "PlanTL-GOB-ES/roberta-base-bne"

tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



textos, tags = [], []
with open(ruta_entrenamiento, 'r', encoding='utf-8') as archivo_csv:
    lector_csv = csv.reader(archivo_csv, delimiter=',')
    next(lector_csv)  # Saltar encabezado
    for fila in lector_csv:
        if fila:
            textos.append(fila[1])
            tags.append(0 if fila[2] == 'left' else 1)

# Dividir los datos en entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(textos, tags, test_size=0.1, random_state=42)

# Crear datasets
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

# Tokenizar los textos
def tokenizar_lote(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

train_dataset = train_dataset.map(tokenizar_lote, batched=True)
val_dataset = val_dataset.map(tokenizar_lote, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Métricas para el entrenador
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

training_args = TrainingArguments(
    output_dir="./resultados",
    eval_strategy="steps",  # Evaluación cada ciertos pasos
    save_strategy="steps",  # Guardar el modelo cada ciertos pasos
    eval_steps=2000,  # Evaluar cada 1000 pasos
    save_steps=2000,  # Guardar cada 1000 pasos
    logging_dir="./logs",
    logging_steps=250,
    learning_rate=3e-5,
    num_train_epochs=0.5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
results = trainer.evaluate()
print("Resultados de evaluación:", results)

# Procesar datos de prueba
textos_a_predecir, ids_a_predecir = [], []
with open(ruta_prueba, 'r', encoding='utf-8') as archivo_csv:
    lector_csv = csv.reader(archivo_csv, delimiter=',')
    next(lector_csv)  # Saltar encabezado
    for fila in lector_csv:
        if fila:
            textos_a_predecir.append(fila[1])
            ids_a_predecir.append(fila[0])

# Crear dataset de prueba
test_dataset = Dataset.from_dict({"text": textos_a_predecir})
test_dataset = test_dataset.map(tokenizar_lote, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Realizar predicciones
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Archivo de salida
with open(archivo_predicciones, 'w', encoding='utf-8', newline='') as archivo_salida:
    escritor_csv = csv.writer(archivo_salida)
    escritor_csv.writerow(["id", "label"])
    for i in range(len(ids_a_predecir)):
        escritor_csv.writerow([ids_a_predecir[i], "left" if preds[i] == 0 else "right"])

print(f"Predicciones guardadas en {archivo_predicciones}")